# Discover EOPF Zarr - Sentinel-2 L2A

## Introduction
This tutorial introduces you to the structure of a Zarr sample for **Sentinel-2 L2A** data. We will demonstrate how to access and open a Zarr product sample with `xarray`, how to visualise the `zarr` encoding structure, explore embedded information, and retrieve relevant metadata for further processing.

### Prerequisites
This tutorial uses a re-processed sample dataset frome the [EOPF Sentinel Zarr Samples Service STAC API](https://stac.browser.user.eopf.eodc.eu/) that is available for download [here](https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s02msil2a/10/products/cpm_v256/S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr). 

The selected `zarr` product is a Sentinel-2 L2A tile from the 10th of June 2025 (file name: `S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr.`).

<hr>

#### Import libraries

In [1]:
import os
import xarray as xr

#### Helper functions

##### `print_gen_structure`
This function helps us to retrieve an visualise the names for each of the stored groups inside a `zarr`. As an output, it will print a general overview of elements inside the `zarr`.

In [2]:
def print_gen_structure(node, indent=""):
    print(f"{indent}{node.name}")     #allows us access each node
    for child_name, child_node in node.children.items(): #loops inside the selected nodes to extract naming
        print_gen_structure(child_node, indent + "  ") # prints the name of the selected nodes

<hr>

## Open a Zarr Store

In a first step, we use the function `open_datatree()` from the `xarray` library to open a Zarr store as a DataTree.<br>
Inside, we ned to define the following key word arguments:<br>

- `filename_or_obj`: path leading to a `zarr` store
- `engine`: `'eopf-zarr'`, designed for the EOPF `zarr` by ESA. 
- `op_mode`: extension by the `xarray-eopf` development for allowing an analysis or native mode. For more information visit the [xarray-eopf](https://eopf-sample-service.github.io/xarray-eopf/) documentation.
- `chunks`: loads the data with dask using the engine’s preferred chunk size, generally identical to the format’s chunk size

The final print of the `DataTree` object is commented out, as the display can be quite extensive, showing the entire content within the Zarr. An alternative is to apply a helper function that only displays the higher level structure as shown in the next code cell.

In [3]:
s2l2a_zarr_sample= xr.open_datatree('https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s02msil2a/10/products/cpm_v256/S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr',
    engine="eopf-zarr", # storage format
    op_mode="native", # no analysis mode
    chunks={}, # allows to open the default chunking
)

If we apply the helper function `print_gen_structure` on the root of the DataTree object, we will get a listing of the tree-like structure of the object. We can see all Zarr groups, such as `measurements`, `quality` and `conditions`, their sub-groups and content.

In [4]:
print("Zarr Sentinel 2 L2A Structure")
print_gen_structure(s2l2a_zarr_sample.root) 
print("-" * 30)

## Extract information from Zarr groups

In a next step, we can explore the content of individual Zarr groups. By specifying the name of the group and subgroup and adding it into square brackets, we can extract the content of the relevant group. Let us for example extract the content of the subgroup `reflectance` under `measurements`.

As a result, it is visible that there are three subgroups of the parent node `measurements/reflectance`: `r10`, `r20` and `r60`, which are the DataArrays with the three different resolutions of the Sentinel-2 L2A data.

The `xarray.DataTree` structure allows the exploration of additional group-related metadata and information. For example, we can find the `chunksize` of each array and the coordinates.

In [5]:
# Retrieving the reflectance groups:
s2l2a_zarr_sample["measurements/reflectance"]

## Extract Zarr metadata on different levels

Through `s2l2a_zarr_sample.attrs[]` we are able to visualise both the `stac_discovery` and `other_metadata` included in the `zarr` store. <br>
<br>
For the properties inside `stac_discovery` for example we can get the parameters included:

In [6]:
# STAC metadata style:
print(list(s2l2a_zarr_sample.attrs["stac_discovery"].keys()))

And from `other_metadata`, we are able to retrieve the information specific to the instrument variables.

In [7]:
# Complementing metadata:
print(list(s2l2a_zarr_sample.attrs["other_metadata"].keys()))

We are also, able to visualise the metadata included at 

## Now it is your turn

As we are able to retrieve several items from the [EOPF Sentinel Zarr Samples Service STAC API](https://stac.browser.user.eopf.eodc.eu/), lets try the following:
<br>
1. Go to the [Sentinel-2 Level-2A collection](https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a)
2. Choose an item of interest.
3. Replicate the workflow and explore the item's `datetime`. When was it retrieved?
4. What are the dimensions?
5. What is the detailed location of the item?

## Conclusion
This tutorial provides an initial understanding of the `zarr` structure for a Sentinel-2 L2A sample.

By using the `xarray` library, we can effectively navigate and inspect the different components within the `zarr` format, including its metadata and array organisation. 
This foundation will help deeply undestand the subsequent data analysis and processing workflows intended in our EOPF 101 series.

In [8]:
s2l2a_zarr_sample[]

In [9]:
print(list(s2l2a_zarr_sample.attrs.keys()))

In [10]:
s2l2a_zarr_sample2= xr.open_dataset('https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s02msil2a/10/products/cpm_v256/S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr',
    engine="eopf-zarr", # storage format
    op_mode="native", # no analysis mode
    chunks={}, # allows to open the default chunking
)

In [11]:
s2l2a_zarr_sample2